# 事前準備

## Googleドライブのマウント

In [207]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 地域コード一覧を作成

兵庫県内の全市区町村リストを作成

In [208]:
import json
from pandas import json_normalize

def df_areas(prefCode):
  with open('drive/MyDrive/statistics-hyogo/resas/arealist.json') as j:
    arealist = json.load(j)
    arealist = list(filter(lambda x: x['prefCode'] == prefCode, arealist))
    arealist = list(filter(lambda x: x['bigCityFlag'] != '1', arealist))
    arealist = list(filter(lambda x: x['governmentType'] == 'city', arealist))

    df = json_normalize(arealist)[['areaCode','areaName']]

    return df

In [196]:
df_hyogo = df_areas(28)
df_hyogo

,areaCode,areaName
0,28100,神戸市
1,28201,姫路市
2,28202,尼崎市
3,28203,明石市
4,28204,西宮市
5,28205,洲本市
6,28206,芦屋市
7,28207,伊丹市
8,28208,相生市
9,28209,豊岡市


# estat-APIからデータを取得する関数

In [103]:
import urllib.parse
import urllib.request
import json
import pandas as pd
import numpy as np

"""
estat-APIのレスポンスを取得する関数
"""
def json_response(params):
  p = params.copy()
  p['appId'] = '724e5b90772a3e9289f41a253e4e7e32438f4fff'
  
  # url生成
  url = 'http://api.e-stat.go.jp/rest/2.1/app/json/getStatsData?'
  url += urllib.parse.urlencode(p)

  with urllib.request.urlopen(url) as response:
    return json.loads(response.read().decode('utf-8'))


"""
estat-APIのレスポンスをDataFrameに変換する
"""
def df_response(response):

  CLASS_OBJ = response['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']

  # times
  dict_times = list(filter(lambda item : item['@id'] == 'time', CLASS_OBJ))[0]['CLASS']
  
  # VALUE
  VALUE = response['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE']
  df = pd.json_normalize(VALUE).replace('-', '0').replace('X', '0').astype({'$': 'int'})

  # 年次を列展開
  for time in dict_times:
    df[time['@name']] = df.apply(lambda x: x['$'] if x['@time'] == time['@code'] else 0, 1)

  # 集計用の列dictを作成
  columns = {t: 'sum' for t in [d.get('@name') for d in dict_times]}
  
  # 集計
  df = df.groupby(['@area','@unit']).agg(columns).reset_index().rename(columns={'@area': 'areaCode', '@unit': '単位'})

  return df

In [ ]:
params = {
    'statsDataId':'0000020201',
    'cdCat01':'A1101',
    }

res = json_response(params)
df = df_response(res)
df

# 兵庫県のデータ抽出

In [199]:
def hyogo(params):

  # estat-APIからデータ取得
  res = json_response(params)
  df = df_response(res)

  # 兵庫県抽出
  df_hyogo = df_areas(28)
  df = pd.merge(df_hyogo, df,on='areaCode', how='left')

  return df

## 総人口


In [203]:
# estat-APIのパラメータ
params = {
    'statsDataId':'0000020201',
    'cdCat01':'A1101',
    }

# estat-APIからデータ取得
df = hyogo(params)

# csvに保存
from google.colab import files
df.to_csv('兵庫県の総人口.csv', encoding="shift_jis")
files.download('兵庫県の総人口.csv')

# 表示
df

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,areaCode,areaName,単位,1980年度,1981年度,1982年度,1983年度,1984年度,1985年度,1986年度,...,2011年度,2012年度,2013年度,2014年度,2015年度,2016年度,2017年度,2018年度,2019年度,2020年度
0,28100,神戸市,人,1367390,0,0,0,0,1410834,0,...,0,0,0,0,1537272,0,0,0,0,0
1,28201,姫路市,人,494825,0,0,0,0,506101,0,...,0,0,0,0,535664,0,0,0,0,0
2,28202,尼崎市,人,523650,0,0,0,0,509115,0,...,0,0,0,0,452563,0,0,0,0,0
3,28203,明石市,人,254869,0,0,0,0,263363,0,...,0,0,0,0,293409,0,0,0,0,0
4,28204,西宮市,人,410329,0,0,0,0,421267,0,...,0,0,0,0,487850,0,0,0,0,0
5,28205,洲本市,人,54826,0,0,0,0,55048,0,...,0,0,0,0,44258,0,0,0,0,0
6,28206,芦屋市,人,81745,0,0,0,0,87127,0,...,0,0,0,0,95350,0,0,0,0,0
7,28207,伊丹市,人,178228,0,0,0,0,182731,0,...,0,0,0,0,196883,0,0,0,0,0
8,28208,相生市,人,41498,0,0,0,0,39868,0,...,0,0,0,0,30129,0,0,0,0,0
9,28209,豊岡市,人,96448,0,0,0,0,96086,0,...,0,0,0,0,82250,0,0,0,0,0


## 総面積

In [204]:
# estat-APIのパラメータ
params = {
    'statsDataId':'0000020202',
    'cdCat01':'B1101',
    }

# estat-APIからデータ取得
df = hyogo(params)

# csvに保存
from google.colab import files
df.to_csv('兵庫県の総面積.csv', encoding="shift_jis")
files.download('兵庫県の総面積.csv')

# 表示
df

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,areaCode,areaName,単位,1980年度,1981年度,1982年度,1983年度,1984年度,1985年度,1986年度,...,2011年度,2012年度,2013年度,2014年度,2015年度,2016年度,2017年度,2018年度,2019年度,2020年度
0,28100,神戸市,ｈａ,0,54286,54291,54357,54358,54417,54544,...,55226,55226,55226,55702,55702,55702,55702,55702,55701,55702
1,28201,姫路市,ｈａ,0,52834,52874,52882,52883,52884,53010,...,53444,53444,53444,53433,53447,53447,53448,53448,53448,53456
2,28202,尼崎市,ｈａ,0,4921,4928,4928,4941,4947,4947,...,4997,5020,5026,5072,5072,5072,5072,5072,5072,5072
3,28203,明石市,ｈａ,0,4926,4929,4929,4928,4928,4941,...,4925,4925,4925,4942,4942,4942,4942,4942,4942,4942
4,28204,西宮市,ｈａ,0,9785,9785,9785,9852,9852,9852,...,9996,9996,9996,9996,9996,9996,9996,9996,9996,9996
5,28205,洲本市,ｈａ,0,18317,18317,18317,18317,18317,18319,...,18248,18248,18248,18238,18238,18238,18238,18238,18238,18238
6,28206,芦屋市,ｈａ,0,1725,1725,1725,1725,1725,1725,...,1847,1847,1847,1847,1847,1847,1847,1847,1847,1847
7,28207,伊丹市,ｈａ,0,2509,2509,2509,2509,2509,2509,...,2497,2497,2497,2500,2500,2500,2500,2500,2500,2500
8,28208,相生市,ｈａ,0,9061,9061,9061,9061,9066,9081,...,9046,9046,9048,9040,9040,9040,9040,9040,9040,9040
9,28209,豊岡市,ｈａ,0,69978,69978,69978,69978,69978,69978,...,69766,69766,69766,69755,69755,69755,69755,69755,69755,69755


## 製造品出荷額

In [205]:
# estat-APIのパラメータ
params = {
    'statsDataId':'0000020203',
    'cdCat01':'C3401',
    }

# estat-APIからデータ取得
df = hyogo(params)

# csvに保存
from google.colab import files
df.to_csv('兵庫県の総面積.csv', encoding="shift_jis")
files.download('兵庫県の総面積.csv')

# 表示
df

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,areaCode,areaName,単位,1980年度,1981年度,1982年度,1983年度,1984年度,1985年度,1986年度,...,2011年度,2012年度,2013年度,2014年度,2015年度,2016年度,2017年度,2018年度,2019年度,2020年度
0,28100,神戸市,百万円,2341038,2539038,2656454,2654352,2842766,2834054,2648533,...,2972253,2866785,2703968,2831802,3125826,3213485,3255637,3439842,3421055,0
1,28201,姫路市,百万円,1618930,1636248,1718172,1691230,1841994,1806583,1606616,...,2077726,2028135,2137607,2408740,2349558,2233199,2357332,2486951,2333912,0
2,28202,尼崎市,百万円,1796629,1848481,1926277,1763067,1842932,1843672,1789415,...,1410339,1347363,1315212,1314443,1377550,1361983,1368173,1449763,1461334,0
3,28203,明石市,百万円,715047,739672,754618,762271,790092,862778,775111,...,1042154,1132734,1057883,1123359,1111685,1149680,1197193,1318994,1385328,0
4,28204,西宮市,百万円,490859,494085,511172,516911,537745,555812,561977,...,365918,264799,298374,310390,307441,310868,289355,291221,267643,0
5,28205,洲本市,百万円,47647,51976,57637,65903,80354,86042,81544,...,71752,75128,72729,65142,285460,50823,45276,46621,44279,0
6,28206,芦屋市,百万円,4256,5096,4958,4793,4468,4882,5443,...,2974,2694,2686,2507,3478,3064,3082,3195,3043,0
7,28207,伊丹市,百万円,614252,649854,642388,726085,889017,820569,680467,...,628025,571903,613212,645384,620425,654349,676823,664570,653874,0
8,28208,相生市,百万円,129366,133479,129973,167859,136011,124590,130112,...,74256,90420,73969,92568,215362,98781,97001,138132,113481,0
9,28209,豊岡市,百万円,92821,100818,96238,95800,101893,108937,108407,...,105483,117641,115824,124783,127999,131109,132962,138749,133966,0


## ごみ排出量

In [206]:
# estat-APIのパラメータ
params = {
    'statsDataId':'0000020208',
    'cdCat01':'H5609',
    }

# estat-APIからデータ取得
df = hyogo(params)

# csvに保存
from google.colab import files
df.to_csv('兵庫県のごみ排出量.csv', encoding="shift_jis")
files.download('兵庫県のごみ排出量.csv')

# 表示
df

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,areaCode,areaName,単位,1980年度,1981年度,1982年度,1983年度,1984年度,1985年度,1986年度,...,2011年度,2012年度,2013年度,2014年度,2015年度,2016年度,2017年度,2018年度,2019年度,2020年度
0,28100,神戸市,ｔ,0,0,0,0,0,0,0,...,587646,582507,572222,563406,559401,547890,545997,547069,539320,0
1,28201,姫路市,ｔ,0,0,0,0,0,0,0,...,194686,194983,194278,194098,183351,186254,185944,186240,186489,0
2,28202,尼崎市,ｔ,0,0,0,0,0,0,0,...,169929,166284,164112,160976,159050,156577,155603,157734,153347,0
3,28203,明石市,ｔ,0,0,0,0,0,0,0,...,109529,109089,108516,108101,107043,104920,104273,104499,102744,0
4,28204,西宮市,ｔ,0,0,0,0,0,0,0,...,183410,181279,179489,177876,177368,174740,173088,173306,169281,0
5,28205,洲本市,ｔ,0,0,0,0,0,0,0,...,19039,18398,18469,18316,17667,18592,16629,16407,16326,0
6,28206,芦屋市,ｔ,0,0,0,0,0,0,0,...,37343,37069,39177,36527,35640,34903,34745,34615,32967,0
7,28207,伊丹市,ｔ,0,0,0,0,0,0,0,...,67843,67507,66797,66644,66957,64898,64142,63998,63361,0
8,28208,相生市,ｔ,0,0,0,0,0,0,0,...,11044,11223,11006,10639,10789,10372,10573,10395,10135,0
9,28209,豊岡市,ｔ,0,0,0,0,0,0,0,...,31356,31223,31123,31195,31303,29858,29395,29120,29053,0


## 世帯

In [209]:
# estat-APIのパラメータ
params = {
    'statsDataId':'0000020201',
    'cdCat01':'A7101',
    }

# estat-APIからデータ取得
df = hyogo(params)

# csvに保存
from google.colab import files
df.to_csv('兵庫県の世帯数.csv', encoding="shift_jis")
files.download('兵庫県の世帯数.csv')

# 表示
df

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,areaCode,areaName,単位,1980年度,1981年度,1982年度,1983年度,1984年度,1985年度,1986年度,...,2011年度,2012年度,2013年度,2014年度,2015年度,2016年度,2017年度,2018年度,2019年度,2020年度
0,28100,神戸市,世帯,458700,0,0,0,0,487849,0,...,0,0,0,0,705459,0,0,0,0,0
1,28201,姫路市,世帯,142940,0,0,0,0,149302,0,...,0,0,0,0,212801,0,0,0,0,0
2,28202,尼崎市,世帯,177486,0,0,0,0,177817,0,...,0,0,0,0,210433,0,0,0,0,0
3,28203,明石市,世帯,77496,0,0,0,0,82288,0,...,0,0,0,0,121890,0,0,0,0,0
4,28204,西宮市,世帯,142167,0,0,0,0,148985,0,...,0,0,0,0,210965,0,0,0,0,0
5,28205,洲本市,世帯,16571,0,0,0,0,17000,0,...,0,0,0,0,18081,0,0,0,0,0
6,28206,芦屋市,世帯,28608,0,0,0,0,30743,0,...,0,0,0,0,41881,0,0,0,0,0
7,28207,伊丹市,世帯,55885,0,0,0,0,58877,0,...,0,0,0,0,78903,0,0,0,0,0
8,28208,相生市,世帯,12428,0,0,0,0,11912,0,...,0,0,0,0,12153,0,0,0,0,0
9,28209,豊岡市,世帯,25465,0,0,0,0,25877,0,...,0,0,0,0,30189,0,0,0,0,0
